In [1]:
import dill
import numpy as np

In [2]:
with open('/home/lucas/tesis2/test_ouputs/sequence_tree_1000samples_20levels_10samples_per_class_5results_per_query.dill', 'rb') as f:
    test_results = dill.load(f)

In [3]:
times = test_results['times']
times_means = {key: np.mean(value) for key, value in times.items()}
times_stds = {key: np.std(value) for key, value in times.items()}

In [4]:
results = test_results['results']

In [5]:
result_means = {}
result_stds = {}
for key, value in results.items():
    values = list(value.values())
    stack = np.vstack(values)
    means = np.mean(stack, axis=0)
    stds = np.std(stack, axis=0)
    result_means[key] = means
    result_stds[key] = stds

In [6]:
result_means

{'BE': array([ 1.        ,  0.8065736 ,  0.7581033 ,  0.71470903,  0.68653811]),
 'CEPH': array([ 1.,  1.,  1.,  1.,  1.]),
 'EB': array([ 1.        ,  0.8065736 ,  0.73463936,  0.66156452,  0.64036643]),
 'LPV': array([ 1.        ,  1.        ,  1.        ,  1.        ,  0.97375898]),
 'ML': array([ 1.        ,  0.84525888,  0.74078362,  0.71711401,  0.68862755]),
 'NV': array([ 1.        ,  0.96131472,  0.97039181,  0.95855701,  0.95087403]),
 'QSO': array([ 1.        ,  0.84525888,  0.69385575,  0.62763775,  0.57152951]),
 'RRL': array([ 1.        ,  0.92262944,  0.87039181,  0.85855701,  0.86399454])}

In [7]:
result_stds

{'BE': array([ 0.        ,  0.1934264 ,  0.22141357,  0.2449047 ,  0.26114718]),
 'CEPH': array([ 0.,  0.,  0.,  0.,  0.]),
 'EB': array([ 0.        ,  0.1934264 ,  0.26536064,  0.27736806,  0.30542676]),
 'LPV': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.05248203]),
 'ML': array([ 0.        ,  0.1895184 ,  0.23766367,  0.26070654,  0.27194537]),
 'NV': array([ 0.        ,  0.11605584,  0.08882457,  0.08470922,  0.09926584]),
 'QSO': array([ 0.        ,  0.1895184 ,  0.16452806,  0.19295657,  0.16017058]),
 'RRL': array([ 0.        ,  0.15474112,  0.21222699,  0.20805212,  0.21188462])}